## Migration vers PostgreSQL

### 1. Préparer le fichier clean :

In [1]:
import pandas as pd
df = pd.read_excel("london_merged_after_preprocessing_pandas.xlsx")
df.describe()

,Nombre de trajets,Température réelle (°C),Température ressentie (°C),Humidité,Vitesse du vent (km/h),is_holiday,is_weekend
count,17414.000000,17414.000000,17414.000000,17414.000000,17414.000000,17414.000000,17414.000000
mean,1143.101642,12.468091,11.520836,0.723250,15.913063,0.022051,0.285403
std,1085.108068,5.571818,6.615145,0.143132,7.894570,0.146854,0.451619
min,0.000000,-1.500000,-6.000000,0.205000,0.000000,0.000000,0.000000
25%,257.000000,8.000000,6.000000,0.630000,10.000000,0.000000,0.000000
50%,844.000000,12.500000,12.500000,0.745000,15.000000,0.000000,0.000000
75%,1671.750000,16.000000,16.000000,0.830000,20.500000,0.000000,1.000000
max,7860.000000,34.000000,34.000000,1.000000,56.500000,1.000000,1.000000


### 2. Charger les variables d’environnement :

In [2]:
import os
import dotenv

dotenv.load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

print(f"DB_HOST: {DB_HOST}\nDB_PORT: {DB_PORT}\nDB_NAME: {DB_NAME}\nDB_USER: {DB_USER}\nDB_PASSWORD: {DB_PASSWORD}")

DB_HOST: localhost
DB_PORT: 5433
DB_NAME: velo_entreprise_db
DB_USER: postgres
DB_PASSWORD: yoora


### 3. Créer la connexion à la base de données avec sqlalchemy :

In [3]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import os

# Encoder le mot de passe pour gérer les caractères spéciaux
password_encoded = quote_plus(DB_PASSWORD)

# Chaîne de connexion
# Fixed version:
engine_url = f"postgresql+psycopg2://{DB_USER}:{password_encoded}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(engine_url)

try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT version();"))
        print("Connexion réussie ! Version PostgreSQL :", result.fetchone()[0])
except Exception as e:
    print("Erreur de connexion :", e)
    exit()  

Connexion réussie ! Version PostgreSQL : PostgreSQL 16.11, compiled by Visual C++ build 1944, 64-bit


### 4. Créer la table et insérer les données :

In [4]:

# Créer la table 
table_name = "london_bikes_final"  # nom de la table dans PostgreSQL

# Define the schema strictly
from sqlalchemy.types import DateTime, Integer, Float, String
column_types = {
    'timestamp': DateTime(),             
    'Nombre de trajets': Integer(),      
    'Température réelle (°C)': Float(),  
    'Température ressentie (°C)': Float(),
    'Humidité': Float(),
    'Vitesse du vent (km/h)': Float(),
    'Météo': String(50),                 
    'Saison': String(20),                
    'is_holiday': Integer(),             
    'is_weekend': Integer()
}
#insérer les données :
try:
    df.to_sql(table_name, engine, if_exists='replace', dtype=column_types, index=False)
    print(f"Données insérées avec succès dans la table '{table_name}' !")
except Exception as e:
    print("Erreur lors de l'insertion :", e)

Données insérées avec succès dans la table 'london_bikes_final' !


### 5. Vérification de l'insertion des données :

In [5]:
try:
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT COUNT(*) FROM {table_name};"))
        total = result.fetchone()[0]
        print(f"Nombre total de lignes dans '{table_name}': {total}")
except Exception as e:
    print("Erreur lors de la vérification :", e)

Nombre total de lignes dans 'london_bikes_final': 17414


## Requêtes SQL significatives :

### 1.1. Total de trajets par météo:

In [10]:

cursor = engine.connect()
results = cursor.execute(text("""SELECT SUM("Nombre de trajets") AS total_de_trajets,
                                    "Météo"
                                FROM public.london_bikes_final
                                GROUP BY "Météo";"""))
output = results.fetchall()

df = pd.DataFrame(output, columns = ["total_de_trajets", "Météo"])
df

,total_de_trajets,Météo
0,15051,Neige
1,6035580,Nuages épars
2,929978,Couvert
3,7146847,Clair
4,1526461,Pluie
5,8168,Pluie avec orage
6,4243887,Nuages fragmentés


### 1.2. Total de trajets par jours  :

In [12]:
query = """SELECT TO_CHAR("timestamp", 'DD') AS "jours de la semaine", 
                    SUM("Nombre de trajets") AS "Total des Traject"
            FROM public.london_bikes_final
            GROUP BY "jours de la semaine"
            ORDER BY "jours de la semaine" ASC  ;
        """
results = cursor.execute(text(query))
output = results.fetchall()

df = pd.DataFrame(output, columns =["jours de la semaine", "Total des Traject"]) 
df

,jours de la semaine,Total des Traject
0,01,619537
1,02,600641
2,03,635968
3,04,658083
4,05,652522
5,06,697667
6,07,684249
7,08,696937
8,09,725225
9,10,665536


### 1.3. Total de trajets par saison :

In [13]:
query = """SELECT SUM("Nombre de trajets") AS total_de_trajets, "Saison"
            FROM public.london_bikes_final
            GROUP BY "Saison";
        """
results = cursor.execute(text(query))
output = results.fetchall()

df = pd.DataFrame(output, columns =["Nombre de trajets", "Saison"]) 
df

,Nombre de trajets,Saison
0,3558087,Hiver
1,4850236,Printemps
2,5073040,Automne
3,6424609,Ete


### 2. Moyenne des trajets par jours :

In [14]:
query = """SELECT  TO_CHAR(timestamp,'DD') AS jours, AVG("Nombre de trajets") AS "Moyenne des Trajets"
            FROM public.london_bikes_final
            GROUP BY jours
            ORDER BY jours ASC;
        """
results = cursor.execute(text(query))
output = results.fetchall()

df = pd.DataFrame(output, columns =["jours","Moyenne des trajets"]) 
df

,jours,Moyenne des trajets
0,01,1077.4556521739130435
1,02,1088.1177536231884058
2,03,1127.6028368794326241
3,04,1144.4921739130434783
4,05,1140.7727272727272727
5,06,1211.2274305555555556
6,07,1194.1518324607329843
7,08,1222.6964912280701754
8,09,1261.2608695652173913
9,10,1157.4539130434782609


### 3. Heures ou jours avec le plus de trajets (Top N)

In [15]:
query = """SELECT "timestamp", Max("Nombre de trajets") AS "Nombre des trajets"
            FROM public.london_bikes_final
            GROUP BY "timestamp"
            ORDER BY "Nombre des trajets" DESC
            LIMIT 3; -- I choose 3 as limit not 1 just for verfication, that
	    	-- to make sure the TOP1 on the list is greater than the 2nd, and the clause works just fine.
        """
results = cursor.execute(text(query))
output = results.fetchall()

df = pd.DataFrame(output, columns =["timestamp", "Nombre de trajets"]) 
df

,timestamp,Nombre de trajets
0,2015-07-09 17:00:00,7860
1,2015-07-09 08:00:00,7531
2,2015-08-06 17:00:00,7208


### 4. Comparaison week-end vs semaine :

In [16]:
query = """SELECT
                Moy_Semaine,
                Moy_Weekend,

                (Moy_Semaine / (Moy_Semaine + Moy_Weekend)) * 100 AS "% Semaine",
                (Moy_Weekend / (Moy_Semaine + Moy_Weekend)) * 100 AS "% Weekend"
            FROM (
                SELECT
                    AVG(CASE WHEN is_weekend = 0 THEN "Nombre de trajets" END) AS Moy_Semaine,
                    AVG(CASE WHEN is_weekend = 1 THEN "Nombre de trajets" END) AS Moy_Weekend
                FROM public.london_bikes_final
                ) AS calculs;
        """
results = cursor.execute(text(query))
output = results.fetchall()

df = pd.DataFrame(output, columns =["Moy_Semaine", "Moy_Weekend","% Semaine", "% Weekend"]) 
df

,Moy_Semaine,Moy_Weekend,% Semaine,% Weekend
0,1209.2748312439729990,977.4156941649899396,55.30159925203910386900,44.69840074796089613100
